# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Изучение данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
from numpy import floor
print((df['calls'] - floor(df['calls'])).unique())
print((df['messages'] - floor(df['messages'])).unique())

[0.]
[0.]


Данные не содеражт пропусков, количество звонков и пропуски можно привести к целочисленному типу

In [5]:
df.calls = pd.to_numeric(df.calls,downcast="integer")
df.messages = pd.to_numeric(df.messages,downcast="integer")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int16  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int16  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int16(2), int64(1)
memory usage: 88.0 KB


## Разбейте данные на выборки

Так как тестовая выборка отдельно не предоставляется, то исходные данные надо разбить на 3 выборки:
- обучающая
- валидационная
- тестовая

в отношении 60/20/20

In [6]:
features = df.drop('is_ultra',axis = 1)
target = df['is_ultra']

features_train, features_valid_test, target_train, target_valid_test = train_test_split(features, target, test_size = 0.4, random_state = 12345)

features_valid, features_test, target_valid, target_test = train_test_split(features_valid_test, target_valid_test, test_size = 0.5, random_state = 12345)

В данных выделен целевой признак: "is_ultra"

In [7]:
print(f'Размер обучающей выборки {features_train.shape, target_train.shape}')
print(f'Размер валидационной выборки{features_valid.shape, target_valid.shape}')
print(f'Размер тестовой выборки{features_test.shape, target_test.shape}')

Размер обучающей выборки ((1928, 4), (1928,))
Размер валидационной выборки((643, 4), (643,))
Размер тестовой выборки((643, 4), (643,))


## Исследуйте модели

### Дерево решений

In [8]:
best_model = None
best_accuracy = 0
for i in range(1,6):
    model = DecisionTreeClassifier(max_depth = i, random_state = 1234)
    model.fit(features_train,target_train)
    predictions_valid = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions_valid)
    print("max_depth = ",i,"accurcy = ",accuracy_score(target_valid, predictions_valid))
    if best_accuracy < accuracy:
        best_model_tree = model
        best_accuracy = accuracy

max_depth =  1 accurcy =  0.7542768273716952
max_depth =  2 accurcy =  0.7822706065318819
max_depth =  3 accurcy =  0.7853810264385692
max_depth =  4 accurcy =  0.7791601866251944
max_depth =  5 accurcy =  0.7791601866251944


**Вывод:** Модель классификации "Дерево решений" дала наилучшей результат с accuracy = 0.785, с гиперпараметром "максимальная глубина" = 3 

### Случайный лес

In [9]:
best_model_forest = None
best_result = 0
for est in range(1,11):
    for i in range(1,6):
        model = RandomForestClassifier(random_state = 12345, n_estimators = est,max_depth = i)
        model.fit(features_train,target_train)
        predictions_valid = model.predict(features_valid)
        result = accuracy_score(predictions_valid,target_valid)
        if result > best_result:
            best_model_forest = model
            best_result = result
print(best_model_forest)
print(best_result)

RandomForestClassifier(max_depth=5, n_estimators=8, random_state=12345)
0.7931570762052877


**Вывод:**
Модель классификации "RandomForest" дала наилучший результат с accuracy = 0.793 и гиперпараметрами количество деревьев - 8, и максимальной глубиной дерева - 5.

### Логистическая регрессия

In [10]:
model = LogisticRegression(random_state = 12345)
model.fit(features_train,target_train)
predictions_valid = model.predict(features_valid)
result = accuracy_score(predictions_valid,target_valid)

print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7107309486780715


Вывод: Модель классификации "LogisticRegression" дала наилучший результат с accuracy = 0.71

## Проверьте модель на тестовой выборке

In [11]:
prediction_test = best_model_forest.predict(features_test)
print(accuracy_score(prediction_test,target_test))

0.7962674961119751


### Проверка модели на адекватность

In [12]:
dummy_model = DummyClassifier(strategy='most_frequent').fit(features_train,target_train)
dummy_predictions = dummy_model.predict(features_valid)

print('Test score: ', accuracy_score(target_valid,dummy_predictions))

Test score:  0.7060653188180405


**Вывод:**
Лучшая модель "RandomForest" оказалась точнее чем константная модель "Dummy", с долей правильных ответов 0.796 и 0.706 соответственно

## Общий вывод:

1. Исходные данные были подготовлены
2. На данных были обучены 3 модели:
    - Дерево решений, доля правильных ответов на валидационной выборке составила 0.785
    - Случайный лес, доля правильных ответов на валидационной выборке составила 0.793
    - Логистическая регрессия, доля правильных ответов на валидационной выборке составила 0.71
3. Наилучшая модель "Случайный лес" на тестовой выборке дала долю правильных ответв: 0.796